In [152]:
import numpy as np
import pandas as pd
import matplotlib as plt
import os

In [153]:
os.getcwd()

'/Users/mihirsaurkar/Documents/projects/ML-apps'

In [2]:
import tensorflow as tf

In [157]:
base_path = os.path.abspath("shakespeare")

print(base_path)

/Users/mihirsaurkar/Documents/projects/ML-apps/shakespeare


In [158]:
path_to_file = os.path.join(base_path,"shakespeare.txt")
text = open(path_to_file,'r').read()

In [5]:
#print(text[40500:41800])

In [6]:
vocab = sorted(set(text))

In [7]:
len(vocab) # will affect the last dense layer

84

In [8]:
for pair in enumerate(vocab):
    print(pair)

(0, '\n')
(1, ' ')
(2, '!')
(3, '"')
(4, '&')
(5, "'")
(6, '(')
(7, ')')
(8, ',')
(9, '-')
(10, '.')
(11, '0')
(12, '1')
(13, '2')
(14, '3')
(15, '4')
(16, '5')
(17, '6')
(18, '7')
(19, '8')
(20, '9')
(21, ':')
(22, ';')
(23, '<')
(24, '>')
(25, '?')
(26, 'A')
(27, 'B')
(28, 'C')
(29, 'D')
(30, 'E')
(31, 'F')
(32, 'G')
(33, 'H')
(34, 'I')
(35, 'J')
(36, 'K')
(37, 'L')
(38, 'M')
(39, 'N')
(40, 'O')
(41, 'P')
(42, 'Q')
(43, 'R')
(44, 'S')
(45, 'T')
(46, 'U')
(47, 'V')
(48, 'W')
(49, 'X')
(50, 'Y')
(51, 'Z')
(52, '[')
(53, ']')
(54, '_')
(55, '`')
(56, 'a')
(57, 'b')
(58, 'c')
(59, 'd')
(60, 'e')
(61, 'f')
(62, 'g')
(63, 'h')
(64, 'i')
(65, 'j')
(66, 'k')
(67, 'l')
(68, 'm')
(69, 'n')
(70, 'o')
(71, 'p')
(72, 'q')
(73, 'r')
(74, 's')
(75, 't')
(76, 'u')
(77, 'v')
(78, 'w')
(79, 'x')
(80, 'y')
(81, 'z')
(82, '|')
(83, '}')


In [9]:
char_to_ind = {char:ind for ind,char in enumerate(vocab)}

In [10]:
char_to_ind['H'] # encoder

33

In [11]:
ind_to_char = np.array(vocab) # decoder

In [12]:
ind_to_char[33]

'H'

In [13]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [14]:
encoded_text.shape

(5445609,)

In [15]:
seq_len = 120 # depends on the structure of text so model can learn the pattern

In [16]:
total_num_seq = len(text) // (seq_len+1)

In [17]:
total_num_seq

45005

In [18]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

Metal device set to: Apple M2 Pro


In [19]:
type(char_dataset)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [20]:
sequences = char_dataset.batch(seq_len+1,drop_remainder = True)

In [21]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

In [22]:
dataset = sequences.map(create_seq_targets)

In [23]:
for input_txt,target_txt in dataset.take(1):
    print(input_txt.numpy())
    print("".join(ind_to_char[input_txt.numpy()]))
    print("\n")
    print(target_txt.numpy())
    print("".join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


2023-04-24 09:21:13.922252: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [24]:
batch_size = 128

In [25]:
buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)


In [26]:
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(128, 120), dtype=tf.int64, name=None), TensorSpec(shape=(128, 120), dtype=tf.int64, name=None))>

In [27]:
vocab_size = len(vocab)

In [28]:
vocab_size

84

In [29]:
embed_dim = 64

In [30]:
rnn_neurons = 1026

In [31]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [32]:
def sparse_cat_loss(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true,y_pred,from_logits = True)

In [80]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,GRU,Dense,Dropout,LSTM
from tensorflow.keras.callbacks import EarlyStopping

In [67]:
def create_model(vocab_size,embed_dim,rnn_neurons,batch_size):
    
    model = Sequential()
    model.add(Embedding(vocab_size,embed_dim,batch_input_shape = [batch_size,None]))
    model.add(GRU(rnn_neurons, return_sequences = True, 
                  stateful=True, recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))
    
    model.compile('adam',loss=sparse_cat_loss)
    
    return model
    

In [92]:
def create_model_m1(vocab_size,embed_dim,rnn_neurons,batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size,embed_dim,batch_input_shape = [batch_size,None]))
    model.add(GRU(rnn_neurons, 
                  activation='tanh',
                  recurrent_activation="sigmoid",
                  return_sequences = True, 
                  stateful=True, recurrent_initializer='glorot_uniform'))
    model.add(Dropout(0.5))
    model.add(GRU(rnn_neurons, activation='tanh',
                   recurrent_activation="sigmoid",
                   return_sequences = True, 
                   stateful=True,
                   recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))
    model.compile('adam',loss=sparse_cat_loss)
    return model
    

In [93]:
model = create_model_m1(vocab_size=vocab_size,
                     embed_dim=embed_dim,
                     rnn_neurons=rnn_neurons,
                     batch_size=batch_size)

In [94]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (128, None, 64)           5376      
                                                                 
 gru_10 (GRU)                (128, None, 1026)         3361176   
                                                                 
 dropout_4 (Dropout)         (128, None, 1026)         0         
                                                                 
 gru_11 (GRU)                (128, None, 1026)         6322212   
                                                                 
 dense_8 (Dense)             (128, None, 84)           86268     
                                                                 
Total params: 9,775,032
Trainable params: 9,775,032
Non-trainable params: 0
_________________________________________________________________


In [95]:
epochs = 45

In [96]:
callback = EarlyStopping(monitor='loss', patience = 3)
model.fit(dataset,epochs=epochs,callbacks = [callback])

Epoch 1/45
351/351 [==============================] - 127s 357ms/step - loss: 2.9503
Epoch 2/45
351/351 [==============================] - 127s 359ms/step - loss: 2.5729
Epoch 3/45
351/351 [==============================] - 126s 357ms/step - loss: 2.5761
Epoch 4/45
351/351 [==============================] - 125s 355ms/step - loss: 2.5022
Epoch 5/45
351/351 [==============================] - 125s 355ms/step - loss: 2.5038
Epoch 6/45
351/351 [==============================] - 125s 355ms/step - loss: 2.4714
Epoch 7/45
351/351 [==============================] - 126s 356ms/step - loss: 2.4501
Epoch 8/45
351/351 [==============================] - 126s 358ms/step - loss: 2.4231
Epoch 9/45
351/351 [==============================] - 125s 356ms/step - loss: 2.4124
Epoch 10/45
351/351 [==============================] - 125s 355ms/step - loss: 2.4048
Epoch 11/45
351/351 [==============================] - 125s 355ms/step - loss: 2.4048
Epoch 12/45
351/351 [==============================] - 125s 355

In [64]:
from tensorflow.keras.models import load_model

In [58]:
model_path = os.path.join(base_path,"shakespeare_gen.h5")
model_loaded  = create_model(vocab_size,embed_dim,rnn_neurons,batch_size=1)
model_loaded.load_weights(base_path+"shakespeare_gen.h5")
model_loaded.build(tf.TensorShape([1,None,64]))

In [97]:
model_trained = create_model_m1(vocab_size,embed_dim,rnn_neurons,batch_size=1)
model_trained.set_weights(model.get_weights())
model_trained.build(tf.TensorShape([1,None,64]))


In [98]:
model_trained.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (1, None, 64)             5376      
                                                                 
 gru_12 (GRU)                (1, None, 1026)           3361176   
                                                                 
 dropout_5 (Dropout)         (1, None, 1026)           0         
                                                                 
 gru_13 (GRU)                (1, None, 1026)           6322212   
                                                                 
 dense_9 (Dense)             (1, None, 84)             86268     
                                                                 
Total params: 9,775,032
Trainable params: 9,775,032
Non-trainable params: 0
_________________________________________________________________


In [61]:
model_loaded.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (1, None, 64)             5376      
                                                                 
 gru_2 (GRU)                 (1, None, 1026)           3361176   
                                                                 
 dense_2 (Dense)             (1, None, 84)             86268     
                                                                 
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [62]:
def generate_text(model,start_seed, gen_size=500, temp=1.0):
    num_generate = gen_size
    input_eval = [char_to_ind[s] for s in start_seed]
    input_eval = tf.expand_dims(input_eval,0)
    text_generated  = []
    temperatur = temp
    model.reset_states()
    for i in range(num_generate):
        
        predictions = model(input_eval)
        
        predictions = tf.squeeze(predictions,0)
        
        predictions = predictions/temperatur
        
        predicted_id = tf.random.categorical(predictions, num_samples =1) [-1,0].numpy()
        
        input_eval = tf.expand_dims([predicted_id],0)
        
        text_generated.append(ind_to_char[predicted_id])
        
    return (start_seed + "".join(text_generated))

In [88]:
print(generate_text(model_trained,"Romeo", gen_size=1000))

Romeonar sf s sherssees rs or when.
     Wr;

  Thre we whe,       whaks. SHE all time together bide, success
    To entertain assembling by this haste,
    And fead to have your will bring her the gross extremes
    Than you both in your compurries.
  VIOLA. My brother, geter make excellent house.  
  Francis, see you; but the fresh a song varlet plack block,
    An army for a snatcast inwords than to draw enough.
    I my will inslaughter to-morrow
    Is took that fears with seeming theme.
    Fear not our clothed;                            Exeunt




SCENE IV.
Anous, and
SICITITES house

Enter an DUKE
 M] M', Bl.
  faks, alks. FF sot kakat,

  tat  k ntt an.
    N at tan, mand jank?
twe. nox dn codp.
 Mu mls I Fept, tamvn memell mil gn' thamlpvey nem, he tod liw bpy potlllnn. K.tUd peten ln  tonqen ty we Gfs eng. n thly fhet tho pteln, Tongit aml anf he ch pnelt ln. Jopnind nonbt,
Sin parby that L. tppping mh rethank mong pmang mep!
 Fom Le llonf fott,
 ' Fening ?     noveven tota

In [99]:
print(generate_text(model_loaded,"romeo", gen_size=1000))

romeoning their fat lamening!  
     sure?
                                                      Exeunt the Threat

Enter non this rite self-and sweet, and thy face be cross'd
    With the mill of his own impy remorse applause
    With ev'ry charrer tires. Nor how come you too late,  
    Consorned in the life three up the thunder,
    And in her pain is on you.  
                                                         What has himself, in despite of a maid-
    I'll stir out thee.
  CLOWN. To sit obey e                  is not to begin
      Well, go to your worviCes and gard no blood to sea
    Thou canst not speak of comfort Kim.
                              her heir to me,
    As is unreigned English.
  Pol. I hope no more accurs'd on his when says he'll usurp their way,  
    And will offend their him. I  hot Buly Brutus, Desdemona?
    Why dost thou stop my mother?  
  WARWICK. Ay, to the maids here is but edded geam,
    And what they should en Beh!
                           